In [19]:
###################################################
import emcee
import numpy as np
import scipy
from astropy.io import fits
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck18 as cosmo
from functools import partial
import time
from astropy.coordinates import SkyCoord
import astropy.units as u
import pyccl as ccl
import clevar
import numpy as np
from astropy.table import Table
import clmm
from clevar.catalog import ClCatalog, MemCatalog
from clevar.match_metrics import recovery
import matplotlib.pyplot as plt
from clevar.match import MembershipMatch
from clevar.match_metrics import distances
from clevar.match_metrics import scaling
from clevar.match import ProximityMatch
from clevar.match_metrics.recovery import ClCatalogFuncs as r_cf
###################################################

# Imports Catalog and creats a table (t_cl) containing Mass ('M'), redshift ('redshift_true') and richness ('NMEM')


t_halo = Table.read("/sps/lsst/groups/clusters/cl_pipeline_project/pywazp_gaussian_catalogs/matched_halo_cluster_catalog.fits")
t_cl = Table.read("/sps/lsst/groups/clusters/cl_pipeline_project/pywazp_gaussian_catalogs/matched_halo_cluster_catalog.fits")

t_halo = t_halo[t_halo['m200c'] > 0.0]
t_cl = t_cl[t_cl['m200c'] > 0.0]
t_cl = t_cl[t_cl['snr'] > 12.0]
Mp, zp = 10**14.3, 0.5



cosmo = ccl.Cosmology(
    Omega_c=0.1109 / 0.71**2,
    Omega_b=0.02258 / 0.71**2,
    h=0.71,
    n_s=0.963,
    sigma8=0.8,
    w0=-1.0,
    wa=0.0,
    transfer_function="boltzmann_camb"
)
display(t_halo)
max_mass = np.max(t_halo['m200c'])
min_mass = np.min(t_halo['m200c'])
# Scientific notation with 2 decimal places
print(f"{max_mass:.2e}, {min_mass:.2e}") 
display(t_cl)


ra,dec,zp,n200,r200_mpc,id,snr,mt_self,mt_other,mt_multi_self,mt_multi_other,mt_frac_self,mt_frac_other,mt_cross,mt_frac_self_max,mt_frac_other_max,m200c,halo_mass,z_h,ra_true,dec_true,NMEM_true
float64,float64,float32,float32,float32,bytes30,float32,bytes13,bytes13,bytes109,bytes109,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64
50.93591472649795,-41.34538693217072,0.05277125,14.847,1.122,143476,5.502,7300151475,7300151475,7300151475,7300151475,0.4725516024109578,0.2653061224489796,7300151475,0.4725516024109578,0.2653061224489796,63046332035245.07,74621860826501.4,0.04803776741027832,50.957678900761394,-41.3448059771583,13
49.49706323474621,-40.242985768257384,0.115748845,8.003,0.968,79349,6.684,43700151442,43700151442,43700151442,43700151442,0.7942936216414986,0.2127659574468085,43700151442,0.7942936216414986,0.2127659574468085,60807967862264.79,69213477135143.664,0.11421597003936768,49.50377592935664,-40.240490667975706,12
51.11978122453714,-39.620245431872505,0.14177611,7.075,0.861,154590,5.357,52300151442,52300151442,52300151442,52300151442,0.6244911071454774,0.20689655172413793,52300151442,0.6244911071454774,0.20689655172413793,58720563188461.98,62974036412214.086,0.1259174346923828,51.06907727982047,-39.64607328475867,16
50.530714428897404,-39.75747779430141,0.16513884,12.868,1.086,6608,13.427,70700151421,70700151421,70700151421,70700151421,0.7217595669711405,0.38461538461538464,70700151421,0.7217595669711405,0.38461538461538464,68254428849397.19,119985568407436.62,0.16475677490234375,50.52541974335929,-39.759854374325954,16
50.538536858117084,-39.068128185514055,0.22151783,28.22,1.026,15444,10.683,78600151411,78600151411,78600151411,78600151411,0.49027272998606314,0.3118279569892473,78600151411,0.49027272998606314,0.3118279569892473,120390041092326.77,148634909566197.2,0.21005773544311523,50.54038986767568,-39.0704158967892,31
50.17085292813019,-39.51418400574235,0.20489573,40.267,1.249,4532,14.847,95400151411,95400151411,95400151411,95400151411,0.8417974135174581,0.37662337662337664,95400151411,0.8417974135174581,0.37662337662337664,222022091351977.47,228279662101633.8,0.2076319456100464,50.170338564932,-39.51566808485035,59
50.63606969044204,-41.40247981147909,0.19032279,71.619,1.663,1341,20.345,130200151411,130200151411,130200151411,130200151411,0.7262666073048916,0.41089108910891087,130200151411,0.7262666073048916,0.41089108910891087,229655937300912.7,263934081687436.62,0.19159162044525146,50.64208660915858,-41.40265859512897,84
51.192978766528704,-40.8759233021217,0.17799073,12.546,1.08,25735,9.285,191900151411,191900151411,191900151411,191900151411,0.8547775541627649,0.4074074074074074,191900151411,0.8547775541627649,0.4074074074074074,116379211308709.86,127931588723380.28,0.18677890300750732,51.19489753953479,-40.87828426780436,24
50.13097168819828,-41.25581068917369,0.20250845,16.688,0.84,33799,8.605,253900151411,253900151411,253900151411,253900151411,0.3668106354029247,0.3611111111111111,253900151411,0.3668106354029247,0.3611111111111111,58288508524169.016,70912063920676.06,0.19323325157165527,50.13040773558871,-41.24880382828121,13


1.88e+15, 1.00e+13


ra,dec,zp,n200,r200_mpc,id,snr,mt_self,mt_other,mt_multi_self,mt_multi_other,mt_frac_self,mt_frac_other,mt_cross,mt_frac_self_max,mt_frac_other_max,m200c,halo_mass,z_h,ra_true,dec_true,NMEM_true
float64,float64,float32,float32,float32,bytes30,float32,bytes13,bytes13,bytes109,bytes109,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64
50.530714428897404,-39.75747779430141,0.16513884,12.868,1.086,6608,13.427,70700151421,70700151421,70700151421,70700151421,0.7217595669711405,0.38461538461538464,70700151421,0.7217595669711405,0.38461538461538464,68254428849397.19,119985568407436.62,0.16475677490234375,50.52541974335929,-39.759854374325954,16
50.17085292813019,-39.51418400574235,0.20489573,40.267,1.249,4532,14.847,95400151411,95400151411,95400151411,95400151411,0.8417974135174581,0.37662337662337664,95400151411,0.8417974135174581,0.37662337662337664,222022091351977.47,228279662101633.8,0.2076319456100464,50.170338564932,-39.51566808485035,59
50.63606969044204,-41.40247981147909,0.19032279,71.619,1.663,1341,20.345,130200151411,130200151411,130200151411,130200151411,0.7262666073048916,0.41089108910891087,130200151411,0.7262666073048916,0.41089108910891087,229655937300912.7,263934081687436.62,0.19159162044525146,50.64208660915858,-41.40265859512897,84
49.97226138364417,-40.73031619826872,0.27807176,24.932,1.225,8171,12.695,349000151382,349000151382,349000151382,349000151382,0.46738683308082246,0.25925925925925924,349000151382,0.46738683308082246,0.25925925925925924,140488448778816.9,162421527739492.97,0.2746037244796753,49.99763790043953,-40.75255486327657,24
50.08448729912273,-40.24338303417428,0.32030836,22.806,1.173,1224,20.849,397900151373,397900151373,397900151373,397900151373,0.6980156503489297,0.18888888888888888,397900151373,0.6980156503489297,0.18888888888888888,137755570256766.2,198075947325295.78,0.3244130611419678,50.086766251266965,-40.243365459795626,40
51.53591402608318,-40.105900748250654,0.37715277,27.895,1.216,8806,12.444,346300151355,346300151355,346300151355,346300151355,0.8020024733684662,0.3368421052631579,346300151355,0.8020024733684662,0.3368421052631579,74438552019290.14,107995057541408.45,0.3791588544845581,51.5347525254958,-40.10519064315011,32
50.74465568477031,-38.94572601222099,0.39192057,20.424,1.01,8737,12.468,357100151355,357100151355,357100151355,357100151355,0.770736357072577,0.3888888888888889,357100151355,0.770736357072577,0.3888888888888889,95549429246557.75,113040273581070.42,0.3812941312789917,50.73925038423129,-38.9417925943923,31
50.41915077056092,-39.58962833579833,0.39053577,38.184,1.141,1163,21.197,388500151355,388500151355,388500151355,388500151355,0.544235270310982,0.3076923076923077,388500151355,0.544235270310982,0.3076923076923077,145162982864405.66,154662656086535.22,0.3911494016647339,50.41967140685286,-39.590857084167055,39
50.584421137619316,-39.60513725388427,0.38361177,32.791,1.079,1336,20.38,481800151355,481800151355,481800151355,481800151355,0.6784470605749148,0.3627450980392157,481800151355,0.6784470605749148,0.3627450980392157,170961414242073.25,225225499893183.12,0.388502836227417,50.58798816965926,-39.60353878243939,41


In [20]:
## Mass-Richness

In [21]:
def prior(params):
    if len(params) == 6:
        A, B, C, D, E, F = params
        if -2<C<2 and -2<E<2 and -2<F<2 and A > 0 and B > 0 and D > 0:
            return True
        else:
            return False
    else:
        A, B, D, E = params
        if -2<E<2 and A > 0 and B > 0 and D > 0:
            return True
        else:
            return False
def _ln_L(params, X, a, b, use_correction):
    # 1. Prior Check
    if not prior(params):
        return -np.inf
    
    # 2. Extract Data & Params
    pop, M, z = np.log(X["n200"]), X["m200c"], X["z_h"]
    
    if len(params) == 6:
        A, B, C, D, E, F = params
        mean_alpha = A + B*np.log(M/Mp) + C*np.log((1+z)/(1+zp))
        sigma = D + E*np.log(M/Mp) + F*np.log((1+z)/(1+zp))
    else:
        A, B, D, E = params
        mean_alpha = A + B*np.log(M/Mp)
        sigma = D + E*np.log(M/Mp)
    
    # 3. Physicality Check (Sigma must be positive for all points)
    if np.any(sigma <= 0):
        return -np.inf
    
    # 4. Standard Log-likelihood calculation
    # Using np.sum for speed and stability
    term1 = -0.5 * np.log(2 * np.pi * sigma**2)
    term2 = -0.5 * ((pop - mean_alpha) / sigma)**2
    ln_L = np.sum(term1 + term2)
    
    # 5. Optional Truncation Correction (Corrected logic)
    if use_correction:
        # P(a < NMEM < b) = 0.5 * [erf((log(b)-mu)/(sig*sqrt2)) - erf((log(a)-mu)/(sig*sqrt2))]
        inv_sig_root2 = 1.0 / (sigma * np.sqrt(2))
        def erf_func(val):
            return scipy.special.erf((np.log(val) - mean_alpha) * inv_sig_root2)
        
        prob_in_bin = 0.5 * (erf_func(b) - erf_func(a))
        # Add the log of the normalization to the total likelihood
        ln_L -= np.sum(np.log(np.clip(prob_in_bin, 1e-10, None)))
        
    return ln_L

def MCMC(richesse_bin, redshift_bin, itt, redshift_params=True, correction=True):
    a, b = richesse_bin
    c, d = redshift_bin
    
    # Selection
    mask = (t_cl["n200"] > a) & (t_cl["n200"] < b) & \
           (t_cl["zp"] > c) & (t_cl["z_h"] < d) 
            
    X = t_cl[mask]
    display(X)
    if len(X) < 10: # Safety check for empty bins
        print(f"Warning: Bin {richesse_bin}, {redshift_bin} has only {len(X)} clusters.")
        return {'mean': np.zeros(6 if redshift_params else 4), 'std': np.zeros(6 if redshift_params else 4)}

    ndim = 6 if redshift_params else 4
    nwalkers = 48 # Use a multiple of 2 or 4 for better threading
    
    # Better initialization
    if redshift_params:
        guess = np.array([3.2, 0.8, 0.1, 0.5, 0.01, 0.01])
    else:
        guess = np.array([3.2, 0.8, 0.5, 0.01])
        
    p0 = [guess + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)]
    
    sampler = emcee.EnsembleSampler(nwalkers, ndim, _ln_L, args=[X, a, b, correction])
    sampler.run_mcmc(p0, itt, progress=True)
    
    # Correct result extraction (Flattening)
    burn_in = int(itt * 0.3)
    unflat_samples = sampler.get_chain(discard=burn_in, flat=False)
    flat_samples = sampler.get_chain(discard=burn_in, flat=True)    
    return {
        'chains': unflat_samples,
        'mean': np.mean(flat_samples, axis=0),
        'std': np.std(flat_samples, axis=0)
    }

In [22]:
### Unbinned relation

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# ------------------------------------------------------------------------------
# Assuming you run MCMC once for all clusters (global), returns 6 parameters
# ------------------------------------------------------------------------------

# Run your MCMC once
res = MCMC(richesse_bin=(20,100), 
          redshift_bin=(0.2,1.0), 
          itt=4000, redshift_params=True, correction=False)

# Extract chains and statistics
chains = res['chains']  # shape: (nsteps, nwalkers, 6) or similar
means = res['mean']     # length 6
stds  = res['std']      # length 6

# Print best-fit values
param_names = ['A','B','C','D','E','F']
for i, name in enumerate(param_names):
    print(f"{name}: mean={means[i]:.4f}, std={stds[i]:.4f}")

# ------------------------------------------------------------------------------
# Plot chains to check convergence
# ------------------------------------------------------------------------------
n_params = 6
nsteps, nwalkers = chains.shape[0], chains.shape[1]

fig, axes = plt.subplots(n_params, 1, figsize=(12, 2*n_params), sharex=True)
for i in range(n_params):
    ax = axes[i]
    for w in range(nwalkers):
        ax.plot(chains[:, w, i], alpha=0.5)
    # Show mean and median lines
    mean_i = np.mean(chains[:, :, i])
    median_i = np.median(chains[:, :, i])
    ax.axhline(mean_i, color='red', linestyle='--', label='Mean')
    ax.axhline(median_i, color='green', linestyle='-.', label='Median')
    ax.set_ylabel(param_names[i])
    ax.legend()
axes[-1].set_xlabel('MCMC step')
plt.tight_layout()
plt.show()


ra,dec,zp,n200,r200_mpc,id,snr,mt_self,mt_other,mt_multi_self,mt_multi_other,mt_frac_self,mt_frac_other,mt_cross,mt_frac_self_max,mt_frac_other_max,m200c,halo_mass,z_h,ra_true,dec_true,NMEM_true
float64,float64,float32,float32,float32,bytes30,float32,bytes13,bytes13,bytes109,bytes109,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,int64
50.17085292813019,-39.51418400574235,0.20489573,40.267,1.249,4532,14.847,95400151411,95400151411,95400151411,95400151411,0.8417974135174581,0.37662337662337664,95400151411,0.8417974135174581,0.37662337662337664,222022091351977.47,228279662101633.8,0.2076319456100464,50.170338564932,-39.51566808485035,59
49.97226138364417,-40.73031619826872,0.27807176,24.932,1.225,8171,12.695,349000151382,349000151382,349000151382,349000151382,0.46738683308082246,0.25925925925925924,349000151382,0.46738683308082246,0.25925925925925924,140488448778816.9,162421527739492.97,0.2746037244796753,49.99763790043953,-40.75255486327657,24
50.08448729912273,-40.24338303417428,0.32030836,22.806,1.173,1224,20.849,397900151373,397900151373,397900151373,397900151373,0.6980156503489297,0.18888888888888888,397900151373,0.6980156503489297,0.18888888888888888,137755570256766.2,198075947325295.78,0.3244130611419678,50.086766251266965,-40.243365459795626,40
51.53591402608318,-40.105900748250654,0.37715277,27.895,1.216,8806,12.444,346300151355,346300151355,346300151355,346300151355,0.8020024733684662,0.3368421052631579,346300151355,0.8020024733684662,0.3368421052631579,74438552019290.14,107995057541408.45,0.3791588544845581,51.5347525254958,-40.10519064315011,32
50.74465568477031,-38.94572601222099,0.39192057,20.424,1.01,8737,12.468,357100151355,357100151355,357100151355,357100151355,0.770736357072577,0.3888888888888889,357100151355,0.770736357072577,0.3888888888888889,95549429246557.75,113040273581070.42,0.3812941312789917,50.73925038423129,-38.9417925943923,31
50.41915077056092,-39.58962833579833,0.39053577,38.184,1.141,1163,21.197,388500151355,388500151355,388500151355,388500151355,0.544235270310982,0.3076923076923077,388500151355,0.544235270310982,0.3076923076923077,145162982864405.66,154662656086535.22,0.3911494016647339,50.41967140685286,-39.590857084167055,39
50.584421137619316,-39.60513725388427,0.38361177,32.791,1.079,1336,20.38,481800151355,481800151355,481800151355,481800151355,0.6784470605749148,0.3627450980392157,481800151355,0.6784470605749148,0.3627450980392157,170961414242073.25,225225499893183.12,0.388502836227417,50.58798816965926,-39.60353878243939,41
50.55517154863386,-41.25893652222252,0.43090564,30.817,1.021,2939,16.666,413000151347,413000151347,413000151347,413000151347,0.8165663705315611,0.29927007299270075,413000151347,0.8165663705315611,0.29927007299270075,141058448784946.5,140431559838647.89,0.42355191707611084,50.5520294656058,-41.26315733561924,44
50.10748056220077,-40.03857332561972,0.43409717,88.558,1.704,47,42.209,649700151338,649700151338,649700151338,649700151338,0.5885704054993235,0.37735849056603776,649700151338,0.5885704054993235,0.37735849056603776,307207389507064.8,291442335859380.3,0.4336901903152466,50.11237813388829,-40.03640816430374,84


100%|██████████| 4000/4000 [00:51<00:00, 77.77it/s]


A: mean=3.7556, std=0.0077
B: mean=0.3206, std=0.0085
C: mean=0.4380, std=0.0375
D: mean=0.3379, std=0.0052
E: mean=0.0276, std=0.0053
F: mean=-0.0020, std=0.0259


In [14]:
from getdist import plots, MCSamples
import numpy as np

# 1. Create the MCSamples object (keep your existing setup)
# samples = MCSamples(samples=flat_samples, names=param_names, labels=param_labels)

# 2. Get Marginalized Statistics (Mean and 1-sigma limits)
stats = samples.getMargeStats()

print(f"{'Parameter':<10} | {'Mean':<10} | {'1-sigma Lower':<15} | {'1-sigma Upper':<15}")
print("-" * 70)

for i, name in enumerate(param_names):
    par = stats.parWithName(name)
    # par.mean is the average; par.limits[0] is the 68% confidence interval
    print(f"{name:<10} | {par.mean:10.4f} | {par.limits[0].lower:15.4f} | {par.limits[0].upper:15.4f}")

# 3. Get the "Best Fit" (Peak of the 1D marginalized distribution)
# In GetDist, we use the sample's internal density estimator for this.
print("\nMarginalized Peak (Mode) / 'Best Fit':")
for i, name in enumerate(param_names):
    # This finds the peak of the KDE for each 1D parameter
    best_val = samples.getInlineLatex(name, limit=1)
    print(f"{name}: {best_val}")

# 4. Generate the Triangle Plot
# This is the most robust way to visualize the "Best Fit"
g = plots.get_subplot_plotter()
g.triangle_plot(samples, 
                filled=True, 
                title_limit=1) # title_limit=1 prints the Mean +/- 1-sigma on top

# If you want to mark the Mean on the plots as a reference:
# markers = {n: samples.getMargeStats().parWithName(n).mean for n in param_names}
# g.triangle_plot(samples, markers=markers, filled=True)

plt.show()

NameError: name 'samples' is not defined

In [ ]:
## Save to sacc

In [34]:
import math
import itertools

import numpy as np

from numcosmo_py import Nc
from numcosmo_py import Ncm

from astropy.table import Table

from astropy.io import fits
from scipy import stats
from typing import Any
import sacc

data_table = t_cl
data_table = data_table[data_table['z_h'] >= 0.2]
data_table = data_table[data_table['z_h'] <= 0.8]
print(data_table.colnames, len(data_table))

area = 439.78986

cluster_z = data_table["z_h"]
cluster_richness = np.log10(data_table["n200"])

N_richness = 5  # number of richness bins
N_z = 4  # number of redshift bins

cluster_counts, z_edges, richness_edges, _ = stats.binned_statistic_2d(
    cluster_z, cluster_richness, None, "count", bins=[N_z, N_richness]
)
print(cluster_counts)
print(np.sum(cluster_counts))
covariance = np.diag(cluster_counts.flatten())
print(np.mean(cluster_z), np.min(cluster_z))
s_count = sacc.Sacc()
bin_z_labels = []
bin_richness_labels = []

survey_name = "cosmodc2_wazp_halos"
s_count.add_tracer("survey", survey_name, area)

for i, z_bin in enumerate(zip(z_edges[:-1], z_edges[1:])):
    lower, upper = z_bin
    bin_z_label = f"bin_z_{i}"
    s_count.add_tracer("bin_z", bin_z_label, lower, upper)
    bin_z_labels.append(bin_z_label)

for i, richness_bin in enumerate(zip(richness_edges[:-1], richness_edges[1:])):
    lower, upper = richness_bin
    bin_richness_label = f"rich_{i}"
    s_count.add_tracer("bin_richness", bin_richness_label, lower, upper)
    bin_richness_labels.append(bin_richness_label)

#  pylint: disable-next=no-member
cluster_count = sacc.standard_types.cluster_counts

counts_and_edges = zip(
    cluster_counts.flatten(), itertools.product(bin_z_labels, bin_richness_labels)
)


for counts, (bin_z_label, bin_richness_label) in counts_and_edges:
    s_count.add_data_point(
        cluster_count, (survey_name, bin_richness_label, bin_z_label), int(counts)
    )
s_count.add_covariance(covariance)
s_count.to_canonical_order()
s_count.save_fits("./outputs_mor/clusters_sacc_file_cov.sacc", overwrite=True)
s_count.save_fits("cluster_sacc_catalog.sacc", overwrite=True)


['halo_id', 'ra_true', 'dec_true', 'redshift_true', 'halo_mass', 'NMEM', 'files', 'baseDC2/target_halo_fof_halo_id', 'm200c', 'r200c', 'skysim_mtID_m200c', 'skysim_mtID_r200c', 'skysim_mtID_halo_id', 'skysim_mtID_dist_deg', 'skysim_mtID_pixel', 'skysim_mtID_ind', 'skysim_mtID_inds', 'skysim_mtMZ_m200c', 'skysim_mtMZ_r200c', 'skysim_mtMZ_halo_id', 'skysim_mtMZ_dist_deg', 'skysim_mtMZ_pixel', 'skysim_mtMZ_ind', 'skysim_mtMZ_halo_mass', 'skysim_mtMZ_redshift_true', 'NMEM_r_star2', 'NMEM_i_star2', 'NMEM_z_star2', 'NMEM_y_star2', 'NMEM_r_star1.5', 'NMEM_i_star1.5', 'NMEM_z_star1.5', 'NMEM_y_star1.5', 'NMEM_z_25.6', 'NMEM_z_26.1'] 22392
[[ 378. 1209.  591.  120.    8.]
 [ 667. 2587. 1026.  195.   14.]
 [ 995. 3809. 1358.  223.   11.]
 [1925. 5391. 1635.  236.   14.]]
22392.0
0.5810012975018891 0.20034563541412354


In [54]:
import numpy as np
import pandas as pd
import GCRCatalogs
from crow import mass_proxy
from crow import ClusterAbundance
import pyarrow.parquet as pq
import pyarrow as pa
import matplotlib.pyplot as plt
from astropy.table import Table
# Imports
import time
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
from scipy.interpolate import interp1d

from crow import ClusterShearProfile
from crow import ClusterAbundance
from crow.recipes.binned_exact import ExactBinnedClusterRecipe
from crow.recipes.binned_grid import GridBinnedClusterRecipe
from crow import completeness_models, mass_proxy, purity_models, kernel
from crow.properties import ClusterProperty

# ---- Core cosmology and HMF (same numbers as your example) ----
hmf = ccl.halos.MassFuncDespali16(mass_def="200c")

cosmo = ccl.Cosmology(
    Omega_c=0.1109 / 0.71**2,
    Omega_b=0.02258 / 0.71**2,
    h=0.71,
    n_s=0.963,
    sigma8=0.8,
    w0=-1.0,
    wa=0.0,
    transfer_function="boltzmann_camb"
)

mass_richness
# ---- Common grid / binning parameters for later cells ----
mass_grid_size = 80
redshift_grid_size = 40
proxy_grid_size = 40
sky_area = 440
mass_interval = (12.5, 15.27)


cl_abundandce = ClusterAbundance(cosmo, hmf)
m_pivot = 14.3
z_pivot = 0.5
z_bins = [(z_edges[i], z_edges[i+1]) for i in range(len(z_edges) -1)]
proxy_bins = [(richness_edges[i], richness_edges[i+1]) for i in range(len(richness_edges) - 1)]
redshift_distribution = kernel.SpectroscopicRedshift()
mass_richness = mass_proxy.MurataUnbinned(m_pivot, z_pivot)

mass_richness.parameters["mu0"] = 4.5215
mass_richness.parameters["mu1"] = 0.5333
mass_richness.parameters["mu2"] = 0.1571

mass_richness.parameters["sigma0"] = 0.2649
mass_richness.parameters["sigma1"] = 0.0048
mass_richness.parameters["sigma2"] = -0.0440

print(z_edges, richness_edges)
# ---- Common grid / binning parameters for later cells ----
mass_grid_size = 80
redshift_grid_size = 40
proxy_grid_size = 40
sky_area = 440
mass_interval = (12.5, 15.0)


cl_abundandce = ClusterAbundance(cosmo, hmf)

recipe_grid_nocomp = GridBinnedClusterRecipe(
    mass_interval=mass_interval,
    cluster_theory=cl_abundandce,
    redshift_distribution=redshift_distribution,
    mass_distribution=mass_richness,
    proxy_grid_size=proxy_grid_size,
    redshift_grid_size=redshift_grid_size,
    mass_grid_size=mass_grid_size,
)
recipe_grid_nocomp.setup()


exact_t0 = time.time()
grid_nocomp_vals  = np.zeros((len(z_edges) -1,len(richness_edges) - 1))

for i, zb in enumerate(z_bins):
    for j, pb in enumerate(proxy_bins):
        grid_nocomp_vals[i,j]  = recipe_grid_nocomp.evaluate_theory_prediction_counts(np.array(zb), np.array(pb), sky_area)

[0.20034564 0.3502579  0.50017017 0.65008244 0.79999471] [1.04139269 1.43457834 1.82776399 2.22094965 2.6141353  3.00732095]


In [55]:
print(grid_nocomp_vals)

[[4.41108707e+03 4.03272047e+02 1.08519734e+01 5.86093946e-03
  9.37653871e-10]
 [8.46322760e+03 6.90913386e+02 1.56067558e+01 7.17491317e-03
  8.19714413e-10]
 [1.21445153e+04 8.71078114e+02 1.60007985e+01 6.09141438e-03
  5.00027293e-10]
 [1.47419217e+04 9.16063630e+02 1.33035670e+01 4.07044959e-03
  2.41516731e-10]]
